### Основное задание

Возьмите датасет https://www.kaggle.com/ajayrana/hymenoptera-data/kernels
1.Обучите на нем модели ResNet 18 и VGG 16 с нуля (5-10 эпох)
2.Обучите на нем модели ResNet 18 и VGG 16 с использованием FineTuning (5-10 эпох)
3.Добавьте аугментацию данных к пункту 2

Сравните качество всех 3 полученных подходов


### Задание со звездочкой
Примените FineTuning ResNet 18 к FashionMnist. Удалось ли увидеть резкое увеличение качества?